# Análisis Exploratorio de Datos

In [4]:
# Importar librerías
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuración
pd.options.display.max_columns = None
pd.options.display.max_rows = 100


# Conexión a la base de datos
load_dotenv('/app/.env')

DW_USER = os.getenv('DW_USER')
DW_PASSWORD = os.getenv('DW_PASSWORD')
DW_HOST = 'datawarehouse'
DW_PORT = '5432'
DW_DB = os.getenv('DW_DB')

connection_string = f"postgresql://{DW_USER}:{DW_PASSWORD}@{DW_HOST}:{DW_PORT}/{DW_DB}"
engine = create_engine(connection_string)

# Probar conexión
with engine.connect() as conn:
    result = pd.read_sql("SELECT current_database(), current_user;", conn)
    print("✅ Conexión exitosa!")
    print(f"📊 Base de datos: {result.iloc[0, 0]}")
    print(f"👤 Usuario: {result.iloc[0, 1]}")

✅ Conexión exitosa!
📊 Base de datos: homicidios_dw
👤 Usuario: dw_user


In [5]:
# Obtener metadatos de las tablas (sin cargar datos)
query_tables = """
SELECT 
    t.table_name,
    COUNT(c.column_name) as num_columnas,
    pg_size_pretty(pg_total_relation_size(quote_ident(t.table_name)::regclass)) as tamaño
FROM information_schema.tables t
LEFT JOIN information_schema.columns c 
    ON t.table_name = c.table_name 
    AND t.table_schema = c.table_schema
WHERE t.table_schema = 'public'
    AND t.table_type = 'BASE TABLE'
GROUP BY t.table_name
ORDER BY t.table_name;
"""

df_tables = pd.read_sql(query_tables, engine)
print("📋 Tablas en el Data Warehouse:\n")
print(df_tables.to_string(index=False))
print(f"\n📊 Total de tablas: {len(df_tables)}")

📋 Tablas en el Data Warehouse:

      table_name  num_columnas  tamaño
dim_departamento             4   40 kB
       dim_fecha            12 1280 kB
   dim_municipio             6  232 kB
        dim_sexo             2   56 kB
         etl_log            10   48 kB
 fact_homicidios             9   58 MB

📊 Total de tablas: 6


In [7]:
# Probar conexión 
try:
    with engine.connect() as conn:
        print("Conexión exitosa")

        # Listar todas las tablas disponibles
        tables_query = """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
        ORDER BY table_name;
        """
        
        tables_df = pd.read_sql(tables_query, conn)
        print("Tablas disponibles en el datawarehouse:")
        for idx, table in enumerate(tables_df['table_name'], 1):
            print(f"{idx}. {table}")
        print(tables_df)
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")



Conexión exitosa
Tablas disponibles en el datawarehouse:
1. dim_departamento
2. dim_fecha
3. dim_municipio
4. dim_sexo
5. etl_log
6. fact_homicidios
7. v_homicidios_por_departamento
8. v_homicidios_por_mes
9. v_homicidios_por_municipio
10. v_homicidios_por_sexo
                      table_name
0               dim_departamento
1                      dim_fecha
2                  dim_municipio
3                       dim_sexo
4                        etl_log
5                fact_homicidios
6  v_homicidios_por_departamento
7           v_homicidios_por_mes
8     v_homicidios_por_municipio
9          v_homicidios_por_sexo


In [10]:
# Función para describir cualquier tabla (equivalente a DESCRIBE)
def describe_table(table_name, engine):
    query = f"""
    SELECT 
        c.column_name,
        c.data_type,
        c.character_maximum_length,
        c.is_nullable,
        c.column_default,
        CASE 
            WHEN pk.column_name IS NOT NULL THEN 'PK'
            WHEN fk.column_name IS NOT NULL THEN 'FK'
            ELSE ''
        END as key_type
    FROM information_schema.columns c
    LEFT JOIN (
        SELECT ku.column_name
        FROM information_schema.table_constraints tc
        JOIN information_schema.key_column_usage ku
            ON tc.constraint_name = ku.constraint_name
        WHERE tc.constraint_type = 'PRIMARY KEY'
            AND tc.table_name = '{table_name}'
    ) pk ON c.column_name = pk.column_name
    LEFT JOIN (
        SELECT ku.column_name
        FROM information_schema.table_constraints tc
        JOIN information_schema.key_column_usage ku
            ON tc.constraint_name = ku.constraint_name
        WHERE tc.constraint_type = 'FOREIGN KEY'
            AND tc.table_name = '{table_name}'
    ) fk ON c.column_name = fk.column_name
    WHERE c.table_name = '{table_name}'
        AND c.table_schema = 'public'
    ORDER BY c.ordinal_position;
    """
    
    df = pd.read_sql(query, engine)
    return df

# Usar la función
print("ESTRUCTURA DE fact_homicidios")
print("="*80)
df_estructura = describe_table('fact_homicidios', engine)
print(df_estructura.to_string(index=False))

ESTRUCTURA DE fact_homicidios
  column_name                   data_type  character_maximum_length is_nullable                                         column_default key_type
homicidio_key                      bigint                       NaN          NO nextval('fact_homicidios_homicidio_key_seq'::regclass)       PK
    fecha_key                     integer                       NaN          NO                                                   None       FK
    cod_depto                     integer                       NaN          NO                                                   None       FK
     cod_mpio                     integer                       NaN          NO                                                   None       FK
     sexo_key                     integer                       NaN          NO                                                   None       FK
         zona           character varying                      50.0         YES                           